<a href="https://colab.research.google.com/github/khmelkoff/BackSlices/blob/master/TraxRuChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install trax

     |████████████████████████████████| 522kB 7.7MB/s 
     |████████████████████████████████| 3.4MB 13.4MB/s 
     |████████████████████████████████| 215kB 52.8MB/s 
     |████████████████████████████████| 368kB 53.5MB/s 
     |████████████████████████████████| 1.2MB 47.3MB/s 
     |████████████████████████████████| 1.8MB 48.4MB/s 
     |████████████████████████████████| 3.8MB 41.5MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 3.2MB 48.7MB/s 
     |████████████████████████████████| 890kB 37.5MB/s 


In [2]:

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import re
# from unicodedata import normalize
# import sentencepiece as spm

import trax
from trax import layers as tl
from trax.supervised import decoding
from trax.supervised import training

import textwrap
wrapper = textwrap.TextWrapper(width=70)

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load and clear the dialogues

In [4]:
data = pd.read_csv('/content/drive/MyDrive/dialogues.tsv', sep='\t')
data.head()

,persona_1_profile,persona_2_profile,dialogue
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...
2,<span class=participant_1>Я купила дом<br />Я ...,<span class=participant_2>Я пою в караоке<br /...,<span class=participant_1>Пользователь 1: Прив...
3,<span class=participant_1>я врач и женат<br />...,<span class=participant_2>Я мальчик<br />Я учу...,<span class=participant_2>Пользователь 2: Здра...
4,<span class=participant_1>Я школьница.<br />Я ...,<span class=participant_2>Я простоват.<br />Лю...,<span class=participant_1>Пользователь 1: Прив...


In [7]:
test = data.dialogue.iloc[0]
print(wrapper.fill(test))

<span class=participant_2>Пользователь 2: Привет) расскажи о
себе</span><br /><span class=participant_1>Пользователь 1: Привет) под
вкусный кофеек настроение поболтать появилось<br />)</span><br /><span
class=participant_2>Пользователь 2: Что читаешь? Мне нравится
классика</span><br /><span class=participant_2>Пользователь 2: Я тоже
люблю пообщаться</span><br /><span class=participant_1>Пользователь 1:
Люблю животных, просто обожаю, как и свою работу)</span><br /><span
class=participant_1>Пользователь 1: Я фантастику люблю</span><br
/><span class=participant_2>Пользователь 2: А я выращиваю
фиалки</span><br /><span class=participant_2>Пользователь 2: И веду
здоровый и активный образ жизни!</span><br /><span
class=participant_1>Пользователь 1: Ух ты, интересно.</span><br
/><span class=participant_2>Пользователь 2: Ты случайно не принц на
белом коне? Я его очень жду<br />..</span><br /><span
class=participant_1>Пользователь 1: А у меня из хобби каждую неделю
тусить с моим лучшим<br />друг

In [8]:
def dialogue_clearing(string):
    '''
    Clear text from special characters
    Args:
        string (string): one dialog text
    Return:
        list of senteces
    '''
    clear_dialogue = []
    for s in string.split('</span>'):
        s = re.sub('<span class=participant_1>', '', s)
        s = re.sub('<span class=participant_2>', '', s)
        s = re.sub(r'^<br />', '', s)
        s = re.sub('<br />', ' ', s)
        s = re.sub(' ,', ',', s)
        s = re.sub(' \.', '.', s)
        s = re.sub(',', ', ', s)
        s = re.sub('  ', ' ', s)
        s = re.sub(r'\r\n', ' ', s)
        s = re.sub('"', '', s)
        
        if len(s) != 0:
            clear_dialogue.append(s)
    
    return clear_dialogue

In [11]:
for s in dialogue_clearing(test):
    print(s)

Пользователь 2: Привет) расскажи о себе
Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 2: Что читаешь? Мне нравится классика
Пользователь 2: Я тоже люблю пообщаться
Пользователь 1: Люблю животных, просто обожаю, как и свою работу)
Пользователь 1: Я фантастику люблю
Пользователь 2: А я выращиваю фиалки
Пользователь 2: И веду здоровый и активный образ жизни!
Пользователь 1: Ух ты, интересно.
Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду..
Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [12]:
def dialogue_concat(sentences_list):
    '''
    Concatenate several consecutive sentences of one user to one sequence
    
    Args:
        sentences_list (list): list of sentences for one dialogue
    Return:
        list of senteces
    '''
    clear_dialogue_concat = [sentences_list[0]]
    for i in range(1, len(sentences_list)):
        s = sentences_list[i]
        if clear_dialogue_concat[-1][:15] == s[:15]:
            clear_dialogue_concat[-1] = clear_dialogue_concat[-1] + ' ' + s[16:] 
        else:
            clear_dialogue_concat.append(s)
            
    return clear_dialogue_concat    

In [13]:
dialogue = dialogue_clearing(test)
for s in dialogue_concat(dialogue):
    print(s)

Пользователь 2: Привет) расскажи о себе
Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 2: Что читаешь? Мне нравится классика Я тоже люблю пообщаться
Пользователь 1: Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
Пользователь 2: А я выращиваю фиалки И веду здоровый и активный образ жизни!
Пользователь 1: Ух ты, интересно.
Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду..
Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [14]:
def swap_users(sentence_list):
    '''
    Swap user2 to user1
    
    Args:
        sentences_list (list): list of sentences for one dialogue
    Return:
        list of senteces
    '''
    dialogue = []
    for s in sentence_list:
        if s.find('Пользователь 2:') == 0:
            s = re.sub('Пользователь 2:', 'Пользователь 1:', s)
            dialogue.append(s)
            continue
    
        if s.find('Пользователь 1:') == 0:
            s = re.sub('Пользователь 1:', 'Пользователь 2:', s)
            dialogue.append(s)
            continue
    
    return dialogue

In [15]:
dialogue = dialogue_clearing(test)
dialogue = dialogue_concat(dialogue)
for s in swap_users(dialogue):
    print(s)

Пользователь 1: Привет) расскажи о себе
Пользователь 2: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 1: Что читаешь? Мне нравится классика Я тоже люблю пообщаться
Пользователь 2: Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
Пользователь 1: А я выращиваю фиалки И веду здоровый и активный образ жизни!
Пользователь 2: Ух ты, интересно.
Пользователь 1: Ты случайно не принц на белом коне? Я его очень жду..
Пользователь 2: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [29]:
# process the raw data
full_text = ''
dialogue_len = []
untokenized_data = []
for i in tqdm(range(data.shape[0])):
    dialogue = data.dialogue.iloc[i]
    
    dialogue = dialogue_clearing(dialogue)
    dialogue = dialogue_concat(dialogue)
    
    if dialogue[0].find('Пользователь 2:') == 0:
        dialogue = swap_users(dialogue)
        
    string = ' ' + ' '.join(dialogue)
    string_len = len(string.split())
    dialogue_len.append(string_len)
    full_text += string + '\n'
    untokenized_data.append(string)
    
# with open('full_text.txt', 'w', encoding='utf-8') as file:
#      file.write(full_text)   

100%|██████████| 10013/10013 [00:04<00:00, 2358.29it/s]


In [30]:
print(wrapper.fill(untokenized_data[0]))

 Пользователь 1: Привет) расскажи о себе Пользователь 2: Привет) под
вкусный кофеек настроение поболтать появилось ) Пользователь 1: Что
читаешь? Мне нравится классика Я тоже люблю пообщаться Пользователь 2:
Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
Пользователь 1: А я выращиваю фиалки И веду здоровый и активный образ
жизни! Пользователь 2: Ух ты, интересно. Пользователь 1: Ты случайно
не принц на белом коне? Я его очень жду.. Пользователь 2: А у меня из
хобби каждую неделю тусить с моим лучшим другом)


In [ ]:
# train tokenizer
# spm.SentencePieceTrainer.train('--input=full_text.txt --pad_id=0 --bos_id=-1 --eos_id=1 --unk_id=2 \
#                                 --model_prefix=bpe --vocab_size=32000 --model_type=bpe')

# Data preprocessing and create the generator

In [58]:
# loading prepared set to save time
untokenized_data = pd.read_csv('/content/drive/MyDrive/dialogues.csv', sep=';', header=None, encoding='utf-8')
untokenized_data = list(untokenized_data.iloc[:,0])
print(wrapper.fill(untokenized_data[0]))

 Пользователь 1: Привет) расскажи о себе Пользователь 2: Привет) под
вкусный кофеек настроение поболтать появилось ) Пользователь 1: Что
читаешь? Мне нравится классика Я тоже люблю пообщаться Пользователь 2:
Люблю животных, просто обожаю, как и свою работу) Я фантастику люблю
Пользователь 1: А я выращиваю фиалки И веду здоровый и активный образ
жизни! Пользователь 2: Ух ты, интересно. Пользователь 1: Ты случайно
не принц на белом коне? Я его очень жду.. Пользователь 2: А у меня из
хобби каждую неделю тусить с моим лучшим другом)


In [59]:
# train/eval split
margin = int(len(untokenized_data)*0.9)
train_dialogues = untokenized_data[:margin]
print('train cases: ', len(train_dialogues))
eval_dialogues = untokenized_data[margin:]
print('eval cases: ', len(eval_dialogues))

train cases:  9011
eval cases:  1002


In [103]:
def data_generator(data, shuffle=True):
    '''
      Input: 
        data - list of dialogues
        shuffle - If True: shuffle the data order
      Output:
        a tuple containing 2 elements:
        article
        summary
    '''
    
    data_lng = len(data) # len(data)
    index_list = [*range(data_lng)] # Create a list with the ordered indexes of sample data

    if shuffle:
        random.shuffle(index_list) # re-shuffle the order
    
    index = 0 # Start with the first element
    while True:
        # Wrap the index each time that we reach the end of the list
        if index >= data_lng:
            index = 0
            if shuffle:
                random.shuffle(index_list) # re-shuffle the order
            
        sample = data[index_list[index]]
        index += 1
        yield(sample, sample) # input and target

# create data streams
def train_data_stream():
    return data_generator(train_dialogues, shuffle=True)

def eval_data_stream():
    return data_generator(eval_dialogues, shuffle=True)

In [78]:
# def stream(data):
#     # loop over the entire data
#     while True:
#         # get a random element
#         d = random.choice(data)
        
#         # yield a tuple pair of identical values 
#         # (i.e. our inputs to the model will also be our targets during training)
#         yield (d, d)

In [91]:
def detokenize(integers):
    s = trax.data.detokenize(
        integers,
        vocab_type='sentencepiece',
        vocab_file='bpe32.model',
        vocab_dir='/content/drive/MyDrive/') # loading pre-trained tokenizer model to save time

    s = re.sub('Пользователь', '\nПользователь', s)

    return s

def tokenize(s):
    inputs =  next(trax.data.tokenize(
        iter([s]),
        vocab_type='sentencepiece',
        vocab_file='bpe32.model',
        vocab_dir='/content/drive/MyDrive/'))
    
    return list(inputs) 
    
vocab_size = trax.data.vocab_size(
    vocab_type='sentencepiece',
    vocab_file='bpe32.model',
    vocab_dir='/content/drive/MyDrive/')

print('vocab size: ', vocab_size)

vocab size:  32000


In [89]:
tokenize('Пользователь')

[13]

In [104]:
# trax allows us to use combinators to generate our data pipeline
data_pipeline = trax.data.Serial(
    
    trax.data.Shuffle(),
    
    # tokenize the data
    trax.data.Tokenize(vocab_type='sentencepiece',
                       vocab_dir='/content/drive/MyDrive/',
                       vocab_file='bpe32.model'),
    
    # filter too long sequences
    trax.data.FilterByLength(2048),
    
    # bucket by length
    trax.data.BucketByLength(boundaries=[128, 256, 512, 1024],
                             batch_sizes=[32, 16, 8, 4, 2]),
    
    # add loss weights but do not add it to the padding tokens (i.e. 0)
    trax.data.AddLossWeights(id_to_mask=0)
)

# apply the data pipeline to our train and eval sets
train_stream = data_pipeline(data_generator(train_dialogues))
eval_stream = data_pipeline(data_generator(eval_dialogues))

In [105]:
# the stream generators will yield (input, target, weights). let's just grab the input for inspection
inp, _, _ = next(train_stream)

# print the shape. format is (batch size, token length)
print("input shape: ", inp.shape)

# detokenize the first element
print(detokenize(inp[0]))

input shape:  (8, 512)

Пользователь 1: Здравствуйте! Как дела? 
Пользователь 2: Здравствуйте. замечательно. приготовил себе салат из кресс- салата. чем вы заняты? 
Пользователь 1: Гоняю пьяных работников )) Я управдом, слежу за порядком, вот только отдохнуть присел. А вы чем занимаетесь ? 
Пользователь 2: Ем салат из свежих овощей и запивают его минеральной водой. Иногда очень скучно одному, сидеть дома, поэтому по вечера гуляю, дышу свежим воздухом. А вы любите прогулки? 
Пользователь 1: Да, но времени на них мало, постоянно в работе. К счастью я люблю свою работу, поэтому не сильно страдаю от нехватки времени на досуг. А вы вегетарианец? 
Пользователь 2: Не то, чтобы вегетарианец. ем иногда мясо, очень редко. я за ЗОЖ А как ваша семья относится к вашей работе, вас наверное постоянно нет дома? 
Пользователь 1: Похвально. Я вот пьяных не люблю, вечно допекают своими выходками в доме, бывало даже дрался с ними. А так я добрый и отзывчивый) Помимо ЗОЖа чем ещё занимаетесь ? 
Пользовател

# Model